In [1]:
options(warn=-1)
require(tseries)
require(ggplot2)
require(zoo)
require(ggfortify)
require(gridExtra)
require(tidyverse)

Loading required package: tseries

Registered S3 method overwritten by 'quantmod':
  method            from
  as.zoo.data.frame zoo 

Loading required package: ggplot2

Loading required package: zoo


Attaching package: ‘zoo’


The following objects are masked from ‘package:base’:

    as.Date, as.Date.numeric


Loading required package: ggfortify

Loading required package: gridExtra

Loading required package: tidyverse

── Attaching packages ─────────────────────────────────────────────────────────────── tidyverse 1.3.1 ──

✔ tibble  3.1.6     ✔ dplyr   1.0.8
✔ tidyr   1.2.0     ✔ stringr 1.4.0
✔ readr   2.0.2     ✔ forcats 0.5.1
✔ purrr   0.3.4     

── Conflicts ────────────────────────────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::combine() masks gridExtra::combine()
✖ dplyr::filter()  masks stats::filter()
✖ dplyr::lag()     masks stats::lag()



Download using R 251 closing prices for the S&P500 index ending on the 30.12.2008 (the last
trading day in 2008). Assume that an investor has invested $ 1,000,000 in this portfolio on the 30.12.2008.

a) Calculate and plot the discrete returns (in %) for this period. Comment on the plot.

b) Using historical simulation, calculate the VaR of the portfolio for h = 1 and h = 10 (as
usual α = 0.99).

c) Fast forward in time, on the 30.12.2015 the investor is holding a $ 1,000,000-worth portfolio invested in the same index . Again using the past 250 returns, calculate the VaR of the portfolio for h = 1 and h = 10 and α = 0.99. Compare with the results in b)

In [68]:
calculate_VaR = function(returns) {
    alpha = 0.99
    sorted_returns = sort(returns)
    index = floor((1 - alpha) * length(sorted_returns))
    Rt = sorted_returns[index]
    Pt = 10^6
    VaR = abs(Rt * Pt)/100
    return(VaR)
}

get_VaR = function(end) {
    snp = fortify(na.omit(get.hist.quote("^GSPC", quote="Close", end=end, retclass="zoo")))
    snp = tail(snp, n=251)
    snp$Returns = c(NA, (snp$Close[-1] - snp$Close[-250])/snp$Close[-250] * 100)
    var = calculate_VaR(snp$Returns)
    return(data.frame(Date=c(end), VaR=c(var), TenDaysVar=c(sqrt(10)*var)))
}

In [71]:
end = "2008-12-31"
v = get_VaR(end)
end = "2015-12-30"
rbind(v, get_VaR(end))

time series ends   2008-12-30
time series ends   2015-12-29


Date,VaR,TenDaysVar
<chr>,<dbl>,<dbl>
2008-12-31,89295.24,282376.4
2015-12-30,31850.97,100721.6


A company holds a portfolio that consists of shares from four DAX (German index) constituents:

Stock - Number of shares

Siemens - 1000

Allianz - 1600

SAP - 700

BMW - 400

In [73]:
end = "2020-12-30"
start = "2020-01-01"
smns = tail(fortify(na.omit(get.hist.quote("SIEGY", quote="Close", start=start, end=end, retclass="zoo"))), n=251)
allnz = tail(fortify(na.omit(get.hist.quote("ALV.DE", quote="Close", start=start, end=end, retclass="zoo"))), n=251)
sap = tail(fortify(na.omit(get.hist.quote("SAP", quote="Close", start=start, end=end, retclass="zoo"))), n=251)
bmw = tail(fortify(na.omit(get.hist.quote("BMW.DE", quote="Close", start=start, end=end, retclass="zoo"))), n=251)

time series starts 2020-01-02
time series ends   2020-12-29
time series starts 2020-01-02
time series ends   2020-12-29
time series starts 2020-01-02
time series ends   2020-12-29
time series starts 2020-01-02
time series ends   2020-12-29


In [75]:
prices = data.frame(Siemens=smns$Close, Allianz=allnz$Close, SAP=sap$Close, bmw=bmw$Close)

In [79]:
prices$SiemensR = c(NA, (prices$Siemens[-1] - prices$Siemens[-250])/prices$Siemens[-250] * 100)
prices$AllianzR = c(NA, (prices$Allianz[-1] - prices$Allianz[-250])/prices$Allianz[-250] * 100)
prices$SAPR = c(NA, (prices$SAP[-1] - prices$SAP[-250])/prices$SAP[-250] * 100)
prices$BMWR = c(NA, (prices$bmw[-1] - prices$bmw[-250])/prices$bmw[-250] * 100)

In [84]:
require(e1071)

Loading required package: e1071



In [109]:
alpha = c(0.01, 0.05, 0.95, 0.99)
q = floor(alpha * length(sorted_returns))

In [118]:
data.frame(
    Ticker=c("Siemens", "Allianz", "SAP", "bmw"),
    Mean=c(
        mean(prices$SiemensR[-1]),
        mean(prices$AllianzR[-1]),
        mean(prices$SAPR[-1]),
        mean(prices$BMWR[-1])
    ),
    SD=c(
        sd(prices$SiemensR[-1]),
        sd(prices$AllianzR[-1]),
        sd(prices$SAPR[-1]),
        sd(prices$BMWR[-1])
    ),
    Skewness = c(
        skewness(prices$SiemensR[-1]),
        skewness(prices$AllianzR[-1]),
        skewness(prices$SAPR[-1]),
        skewness(prices$BMWR[-1])
    ),
    Kurtosis = c(
        kurtosis(prices$SiemensR[-1]),
        kurtosis(prices$AllianzR[-1]),
        kurtosis(prices$SAPR[-1]),
        kurtosis(prices$BMWR[-1])
    ),
    "Q1"=c(
        sort(prices$SiemensR)[q[1]],
        sort(prices$AllianzR)[q[1]],
        sort(prices$SAPR)[q[1]],
        sort(prices$BMWR)[q[1]]
    ),
    "Q5"=c(
        sort(prices$SiemensR)[q[2]],
        sort(prices$AllianzR)[q[2]],
        sort(prices$SAPR)[q[2]],
        sort(prices$BMWR)[q[2]]
    ),
    "Q95"=c(
        sort(prices$SiemensR)[q[3]],
        sort(prices$AllianzR)[q[3]],
        sort(prices$SAPR)[q[3]],
        sort(prices$BMWR)[q[3]]
    ),
    "Q99"=c(
        sort(prices$SiemensR)[q[4]],
        sort(prices$AllianzR)[q[4]],
        sort(prices$SAPR)[q[4]],
        sort(prices$BMWR)[q[4]]
    ),
    Min=c(
        min(prices$SiemensR[-1]),
        min(prices$AllianzR[-1]),
        min(prices$SAPR[-1]),
        min(prices$BMWR[-1])
    ),
    Max=c(
        max(prices$SiemensR[-1]),
        max(prices$AllianzR[-1]),
        max(prices$SAPR[-1]),
        max(prices$BMWR[-1])
    )
)

Ticker,Mean,SD,Skewness,Kurtosis,Q1,Q5,Q95,Q99,Min,Max
<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
Siemens,0.075209828,2.808110,-0.71841246,4.463539,-12.142372,-4.816994,4.420645,6.837609,-12.47525,11.098892
Allianz,0.008825392,2.746200,0.21902093,9.592558,-9.869317,-3.479237,3.331482,8.511627,-15.32771,15.803850
SAP,0.024013749,2.815288,-2.40180385,18.631118,-10.023074,-4.200690,3.771755,6.483506,-23.15606,8.317667
bmw,0.043490904,2.867554,-0.02617656,5.504235,-11.734915,-4.588342,4.162860,7.717037,-12.97137,14.472367


In [125]:
get_weights = function(day, prices, shares, share_index) {
    total = 0
    for(index in seq(1:4)) {
        total = total + shares[index] * prices[day, index]
    }    
    
    return (shares[share_index] * prices[day, share_index]/total)
}

In [133]:
m = matrix(nrow=251, ncol=4)
shares = c(1000, 1600, 700, 400)
for(day in seq(1:251)) {
    for(stock in seq(1:4)) {
        m[day, stock] = get_weights(day, prices, shares, stock)
    }    
}